In [149]:
import pandas as pd
import glob
import requests
import json
import time

# Project notes

- Identify transactions that are missing from OC that are on any of the accounts (meaning banks, stripe, wise, paypal)
- Identify any transactions on the platform that are missing from the accounts
- Prep a balance/p&l with minimal wrangling

Project steps:

- Clarify project intention and goal (fill in project assessment in Make OC)
- Map out data 
- Write scripts 
- Analyze and prepare accounts

# Load config

In [150]:
# host variables
host = 'opensource'
dateFrom = '2022-12-31T22:59:59.999Z'
dateTo = '2023-12-31T22:59:59.999Z'
year = '2023'

In [151]:
# load API tokens from .env file
import os
from dotenv import load_dotenv
load_dotenv()
octoken = os.getenv('OCTOKEN')
wisetoken = os.getenv('WISETOKEN')

# set the profile id of the target Wise profile
wiseprofile = os.getenv('WISEPROFILE')

In [152]:
# Reload the data or not - set these to false after the first run to avoid reloading
reloadOC = False
reloadWise = False
reloadStripe = False

# Get data

TODO: Query below returns null for all merchandId. Need to get merchantId for all transactions where available to reconcile with Stripe, Wise and Paypal.

In [153]:
# load transactions from open collective or from local file

def fetch_transactions():
    url = f"https://opencollective.com/api/graphql/v2?personalToken={octoken}"

    # Your GraphQL query
    query = """
    query TransactionsTable(
        $hostAccount: AccountReferenceInput,
        $limit: Int!,
        $offset: Int!,
        $type: TransactionType,
        $paymentMethodType: [PaymentMethodType],
        $dateFrom: DateTime,
        $dateTo: DateTime,
        $kind: [TransactionKind],
        $includeIncognitoTransactions: Boolean,
        $includeGiftCardTransactions: Boolean,
        $includeChildrenTransactions: Boolean,
        $virtualCard: [VirtualCardReferenceInput],
        $orderBy: ChronologicalOrderInput,
        $group: String,
        $includeHost: Boolean,
        $expense: ExpenseReferenceInput,
        $order: OrderReferenceInput
      ) {
        transactions(
          host: $hostAccount
          limit: $limit
          offset: $offset
          type: $type
          paymentMethodType: $paymentMethodType
          dateFrom: $dateFrom
          dateTo: $dateTo
          kind: $kind
          includeIncognitoTransactions: $includeIncognitoTransactions
          includeGiftCardTransactions: $includeGiftCardTransactions
          includeChildrenTransactions: $includeChildrenTransactions
          includeDebts: true
          virtualCard: $virtualCard
          orderBy: $orderBy
          group: $group
          includeHost: $includeHost
          expense: $expense
          order: $order
        ) {
          ...TransactionsTableQueryCollectionFragment
          __typename
        }
      }
      
      fragment TransactionsTableQueryCollectionFragment on TransactionCollection {
        totalCount
        offset
        limit
        nodes {
          id
          uuid
          kind
          amount {
            currency
            valueInCents
            __typename
          }
          amountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          netAmount {
            currency
            valueInCents
            __typename
          }
          netAmountInHostCurrency {
            currency
            valueInCents
            __typename
          }
          paymentProcessorFee {
            currency
            valueInCents
            __typename
          }
          paymentMethod {
            name
            service
            sourcePaymentMethod {
              id
              name
              service
              __typename
            }
            type
          }
          giftCardEmitterAccount {  
            id
            name
            slug
            __typename
          }
          group
          type
          description
          createdAt
          merchantId
          isRefunded
          isRefund
          refundTransaction {
            id
          }
          isOrderRejected
          account {
            ... on AccountWithParent {
              parent {
                id
                slug
                name
              }
            }
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          oppositeAccount {
            id
            legacyId
            name
            slug
            isIncognito
            type
            __typename
          }
          expense {
            id
            type
            description
            invoiceInfo 
            tags
            payee {
              id
              name
              slug
              type
              __typename
            }
            __typename
          }
          permissions {
            id
            canRefund
            canDownloadInvoice
            canReject
            __typename
          }
          __typename
        }
        __typename
      }
    """

    headers = {
        "Authorization": f"Bearer {octoken}",
        "Content-Type": "application/json"
    }

    # Initial variables setup
    variables = {
        "hostAccount": {"slug": host},
        "includeIncognitoTransactions": True,
        "includeChildrenTransactions": True,
        "limit": 10000,  # Adjust if necessary but keep a sensible number to avoid server strain
        "offset": 0,  # Will be adjusted for each subsequent request
        "dateFrom": dateFrom,
        "dateTo": dateTo,
        "orderBy": {"field": "CREATED_AT", "direction": "DESC"},
        "includeHost": True
    }

    all_transactions = []  # To hold all transactions
    while True:
        
        # Try catch around json_data = response.json() to avoid errors

        max_retries = 10  # Setting the maximum number of retries
        retries = 0  # Initial retry count

        while retries < max_retries:
            try:
                # Make the HTTP request
                response = requests.post(
                    url, json={'query': query, 'variables': variables}, headers=headers)
                json_data = response.json()
                break 
            except:
                print("Request failed. Attempt:", retries + 1)
                print(response)
                retries += 1 
                time.sleep(10)

        if retries == max_retries:
            print("Maximum retry attempts reached. Exiting.")
        # Extract data
        transactions = json_data['data']['transactions']['nodes']
        all_transactions.extend(transactions)

        # Pagination: Update offset
        variables['offset'] += variables['limit']

        # Check if all transactions are fetched
        if len(all_transactions) >= json_data['data']['transactions']['totalCount']:
            break
        
        # print progress
        print(f'Fetched {len(all_transactions)} transactions')
        # sleep for 10 seconds to avoid server strain
        time.sleep(5)

    return all_transactions

# if account has property parent, replace account with parent
def replace_account_with_parent(transaction):
  if 'parent' in transaction['account']:
    transaction['account'] = transaction['account']['parent']
  return transaction

# apply the function to all transactions
def post_process_transactions(all_transactions):
  return list(map(replace_account_with_parent, all_transactions))

if reloadOC:
  all_transactions = fetch_transactions()
  # dump all transactions to a json file
  post_process_transactions(all_transactions)
  with open(f'data/{host}/{year}/{host}_{year}_all_platform_transactions.json', 'w') as f:
      json.dump(all_transactions, f, indent=2)
  # convert the json file to a dataframe
  df_platform_transactions = pd.json_normalize(all_transactions)
  # save dataframe as pickle
  df_platform_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')
else:
  df_platform_transactions = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')


In [154]:
# get unique transaction kind
transaction_kinds = df_platform_transactions['kind'].unique()

# sample 2 transactions for each kind
df_sample = df_platform_transactions.groupby('kind').apply(lambda x: x.sample(2))

df_sample

id  \
kind                                                                         
ADDED_FUNDS                   100932  0d5a1901-fcf8-4c6a-ad1b-e1b404a9e915   
                              228758  5508b0c7-fb23-4a01-8cdf-dda20dbb68c4   
BALANCE_TRANSFER              736325  330fff1e-1a1b-4ecf-93b2-83e41a39d0b1   
                              671107  ebfec948-939c-43c7-aab5-e724c6ac9b87   
CONTRIBUTION                  461105  5e3d4581-a56c-4cef-91ce-aa00f9cbbea3   
                              115411  0829d135-0e65-4b5b-a051-a95f12f5947c   
EXPENSE                       451765  2c5c3aa9-921f-4726-ac0d-e52dc14eb741   
                              318814  de72e2ca-3bc6-47ba-bfd1-1e806269523e   
HOST_FEE                      269842  367157fc-989e-487e-9638-9bf9eda90c3e   
                              460528  898b06db-c1a1-4012-a9b7-c2e4d5a34eb3   
HOST_FEE_SHARE                210349  5d5e6bfb-44e9-45f6-acb4-a3a408b76ca5   
                              184107  7c6eff92-f595-4eb9-ab31-9dabcbd068cd   
HOST_FEE_SHARE_DEBT           518726  345aea02-4c3c-46d4-b938-257a8eaa924b   
                              597308  4a60a5c2-6679-4daf-b7af-9f141b777eea   
PAYMENT_PROCESSOR_COVER       165996  0d51cb9c-378e-495d-9a52-30e5a54e496c   
                              768347  1fccd73c-5983-4061-8516-e97979aa0025   
PAYMENT_PROCESSOR_DISPUTE_FEE 533212  7d7d2aa1-2892-4304-a933-d0041e8b3dee   
                              395204  14f19031-e7be-4a51-a8f3-ed02d3ad0f5e   
PLATFORM_TIP_DEBT             528121  616d5d8f-a92c-47ed-9ebc-81187ed4ce76   
                              595616  0f077f0f-e40e-41fd-9d0a-93bf4c32faa0   
PREPAID_PAYMENT_METHOD        303270  d4397943-3ba7-4104-85c7-792d9b4eb8ea   
                              511643  ce1f1344-508e-4ce2-b193-7fbda68f407c   

                                                                      uuid  \
kind                                                                         
ADDED_FUNDS                   100932  0d5a1901-fcf8-4c6a-ad1b-e1b404a9e915   
                              228758  5508b0c7-fb23-4a01-8cdf-dda20dbb68c4   
BALANCE_TRANSFER              736325  330fff1e-1a1b-4ecf-93b2-83e41a39d0b1   
                              671107  ebfec948-939c-43c7-aab5-e724c6ac9b87   
CONTRIBUTION                  461105  5e3d4581-a56c-4cef-91ce-aa00f9cbbea3   
                              115411  0829d135-0e65-4b5b-a051-a95f12f5947c   
EXPENSE                       451765  2c5c3aa9-921f-4726-ac0d-e52dc14eb741   
                              318814  de72e2ca-3bc6-47ba-bfd1-1e806269523e   
HOST_FEE                      269842  367157fc-989e-487e-9638-9bf9eda90c3e   
                              460528  898b06db-c1a1-4012-a9b7-c2e4d5a34eb3   
HOST_FEE_SHARE                210349  5d5e6bfb-44e9-45f6-acb4-a3a408b76ca5   
                              184107  7c6eff92-f595-4eb9-ab31-9dabcbd068cd   
HOST_FEE_SHARE_DEBT           518726  345aea02-4c3c-46d4-b938-257a8eaa924b   
                              597308  4a60a5c2-6679-4daf-b7af-9f141b777eea   
PAYMENT_PROCESSOR_COVER       165996  0d51cb9c-378e-495d-9a52-30e5a54e496c   
                              768347  1fccd73c-5983-4061-8516-e97979aa0025   
PAYMENT_PROCESSOR_DISPUTE_FEE 533212  7d7d2aa1-2892-4304-a933-d0041e8b3dee   
                              395204  14f19031-e7be-4a51-a8f3-ed02d3ad0f5e   
PLATFORM_TIP_DEBT             528121  616d5d8f-a92c-47ed-9ebc-81187ed4ce76   
                              595616  0f077f0f-e40e-41fd-9d0a-93bf4c32faa0   
PREPAID_PAYMENT_METHOD        303270  d4397943-3ba7-4104-85c7-792d9b4eb8ea   
                              511643  ce1f1344-508e-4ce2-b193-7fbda68f407c   

                                                               kind  \
kind                                                                  
ADDED_FUNDS                   100932                    ADDED_FUNDS   
                              228758                    ADDED_FUNDS   
BALANCE_TRANSFER              736325      

In [155]:
# load transactions from wise

# Define the base URL for the Wise API
BASE_URL = 'https://api.transferwise.com/v1/transfers'

def fetch_all_transfers(profile_id=None, status=None, source_currency=None, 
                        target_currency=None, created_date_start=None, 
                        created_date_end=None, limit=100):
    headers = {
        'Authorization': f'Bearer {wisetoken}'
    }
    offset = 0
    all_transfers = []

    while True:
        # Prepare query parameters
        query_params = {'limit': limit, 'offset': offset}
        if profile_id:
            query_params['profile'] = profile_id
        if status:
            query_params['status'] = status
        if source_currency:
            query_params['sourceCurrency'] = source_currency
        if target_currency:
            query_params['targetCurrency'] = target_currency
        if created_date_start:
            query_params['createdDateStart'] = created_date_start
        if created_date_end:
            query_params['createdDateEnd'] = created_date_end

        # Make the GET request to the Wise API
        response = requests.get(BASE_URL, headers=headers, params=query_params)
        data = response.json()

        print(data)

        # Break if there are no more results
        if not data:
            break

        for transfer in data:
            all_transfers.append({
                'id': transfer['id'],
                'user': transfer['user'],
                'targetAccount': transfer['targetAccount'],
                'sourceAccount': transfer['sourceAccount'],
                'quote': transfer['quote'],
                'quoteUuid': transfer['quoteUuid'],
                'status': transfer['status'],
                'reference': transfer['reference'],
                'rate': transfer['rate'],
                'created': transfer['created'],
                'business': transfer['business'],
                'transferRequest': transfer['transferRequest'],
                'details.reference': transfer.get('details', {}).get('reference'),
                'hasActiveIssues': transfer['hasActiveIssues'],
                'sourceCurrency': transfer['sourceCurrency'],
                'sourceValue': transfer['sourceValue'],
                'targetCurrency': transfer['targetCurrency'],
                'targetValue': transfer['targetValue'],
                'customerTransactionId': transfer['customerTransactionId']
            })

        # Update the offset for the next iteration
        offset += limit

    return all_transfers

if reloadWise:
    transfers = fetch_all_transfers(profile_id=wiseprofile, created_date_start=dateFrom, created_date_end=dateTo)

    # create transfers dataframe
    df_wise_transactions = pd.DataFrame()

    for transfer in transfers:
        # add to dataframe with concat
        df_wise_transactions = pd.concat([df_wise_transactions, pd.DataFrame([transfer])])

    # save dataframe as pickle
    df_wise_transactions.to_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

In [156]:
# load transactions from stripe export
if reloadStripe:
    df_stripe_in = pd.read_csv(f'data/{host}/{year}/{host}_{year}_stripe.csv')
    # ensure only relevant columns
    df_stripe_in = df_stripe_in[[
    'id',
    'Description',
    'Seller Message',
    'Amount',
    'Amount Refunded',
    'Currency',
    'Converted Amount',
    'Converted Amount Refunded',
    'Fee',
    'Taxes On Fee',
    'Converted Currency',
    'Mode',
    'Status',
    'Statement Descriptor',
    'Customer ID',
    'Customer Description',
    'Captured',
    'Card ID',
    'PaymentIntent ID',
    'Application Fee',
    'Application ID',
    'from (metadata)',
    'to (metadata)'
    ]]
    df_stripe_in.to_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')
    


In [157]:
# load open collective data
df_platform = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_all_platform_transactions.pkl')

# load stripe data
df_stripe = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_stripe_transactions.pkl')

# load wise data
df_wise = pd.read_pickle(f'data/{host}/{year}/df_{host}_{year}_wise_transactions.pkl')

# load bank data
dir = f'data/{host}/{year}/oscbank_1'
df_bank_1 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_1 = df_bank_1.dropna(axis=1, how='all')

dir = f'data/{host}/{year}/oscbank_2'
df_bank_2 = pd.concat([pd.read_csv(f) for f in glob.glob(f'{dir}/*.csv')], ignore_index=True)
df_bank_2 = df_bank_2.dropna(axis=1, how='all')

# Post process data

In [158]:
# post process

# remove failed and pending transactions from Stripe
df_stripe = df_stripe[df_stripe['Status'] != 'Failed']
df_stripe = df_stripe[df_stripe['Status'] != 'Pending']

# remove cancelled from Wise
df_wise = df_wise[df_wise['status'] != 'cancelled']

# Inspect transactions dataframe

In [159]:
# show all columns
pd.set_option('display.max_columns', None)
df_platform.sample(10)

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
146013,09e6fc7f-cf99-4a09-a508-2f40bbf499fc,09e6fc7f-cf99-4a09-a508-2f40bbf499fc,CONTRIBUTION,NaN,NaN,e2c934d6-1044-4ab1-adea-90032f6706ed,CREDIT,Monthly financial contribution to LinuxServer ...,2023-11-01T02:10:36.091Z,ch_3O7Tt7BYycQg1OMf1WCOXQAQ,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,5ax8emk7-lnw54q7x-bow6gyvj-0o93zdrb,52019.0,LinuxServer,linuxserver,False,COLLECTIVE,Collective,lk9mbw7y-48r3zq35-5vn60ej5-lavnodgx,58014,MountainMaster,mountainmaster,False,INDIVIDUAL,Individual,zaxon793-jy8gpld0-ax96brkd-emwl5v04,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612850,48d56130-dd23-4b45-abd6-d6faefe31c2d,48d56130-dd23-4b45-abd6-d6faefe31c2d,HOST_FEE,NaN,NaN,9966a38c-ef48-4b05-9aa1-77c083e0e8d8,DEBIT,Host Fee,2023-04-02T06:11:46.218Z,None,False,False,NaN,False,NaN,Debit,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,0,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004,Open Source Collective,opensource,False,ORGANIZATION,Organization,3z8arxve-ymko60nw-8mbpgl5n-bj9w704d,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215073,71adef05-0d87-4c15-9743-bc2c8afb34b3,71adef05-0d87-4c15-9743-bc2c8afb34b3,HOST_FEE_SHARE,NaN,NaN,90fd9f09-a508-4dec-a1dd-e8485548332f,DEBIT,Host Fee Share,2023-10-01T05:04:53.568Z,None,False,False,NaN,False,NaN,Debit,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,mlo94zn7-x08dpovd-50o6ewga-3vjbrky5,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747376,8b4d2e72-77df-4afb-b3b9-1a8dfbb04ec5,8b4d2e72-77df-4afb-b3b9-1a8dfbb04ec5,HOST_FEE,NaN,NaN,88f87188-a896-4937-8fcf-bc8e56311b3f,CREDIT,Host Fee,2023-02-02T09:10:01.465Z,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914,Logseq,logseq,False,COLLECTIVE,Collective,rxg0j35l-zkwm6vmb-xwx6voe9-8n47daby,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [160]:
# count transactions by kind
df_platform.groupby(['kind']).size()

kind
ADDED_FUNDS                        2616
BALANCE_TRANSFER                    116
CONTRIBUTION                     193827
EXPENSE                            7582
HOST_FEE                         385894
HOST_FEE_SHARE                   192946
HOST_FEE_SHARE_DEBT               61479
PAYMENT_PROCESSOR_COVER             728
PAYMENT_PROCESSOR_DISPUTE_FEE        18
PLATFORM_TIP_DEBT                    11
PREPAID_PAYMENT_METHOD               13
dtype: int64

# Reconciling Stripe

TODO: Before we can do this, we need to get the merchantIDs.

In [161]:
# copy df_strip to df_s
df_stripe_reconciliation = df_stripe.copy()

In [162]:
df_stripe_reconciliation.head()

,id,Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata)
0,ch_3OTXgHBYycQg1OMf0IfKN8Sw,Monthly financial contribution to Logseq (Back...,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI7wZ2sLFYmMPi,https://opencollective.com/sanket-sharma,True,pm_1OTXmVBYycQg1OMfzXUwCvAm,pi_3OTXgHBYycQg1OMf0PkaNI3M,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/sanket-sharma,https://opencollective.com/logseq
3,ch_3OTXVMBYycQg1OMf0qVZ5tjm,Financial contribution to OpenSCAD,Payment complete.,50.00,0.0,usd,50.00,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTXVpBYycQg1OMfien2j8Cr,pi_3OTXVMBYycQg1OMf02f4Vhyl,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/openscad
4,ch_3OTULSBYycQg1OMf10wlDceK,Monthly financial contribution to LibreELEC,Payment complete.,10.00,0.0,usd,10.00,0.0,1.24,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI4UZkic5DVDO1,https://opencollective.com/daniel-dolejska,True,pm_1OTULSBYycQg1OMf7Lt0BwEi,pi_3OTULSBYycQg1OMf1saV7WWG,0.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/daniel-dolejska,https://opencollective.com/libreelec
5,ch_3OTUG8BYycQg1OMf0FRuZULZ,Financial contribution to Spotube,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTUI5BYycQg1OMfAbkDmXhS,pi_3OTUG8BYycQg1OMf0AcGFa2r,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/spotube
7,ch_3OTT8EBYycQg1OMf0gR3PzrY,Financial contribution to .fmbot (.fmbot lifet...,Payment complete.,49.99,0.0,usd,49.99,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTT94BYycQg1OMf9AOHyAVx,pi_3OTT8EBYycQg1OMf0xdn31JV,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/fmbot


In [163]:
# df_platform where kind is CONTRIBUTION and paymentMethod.service is STRIPE
df_platform_stripe = df_platform[(df_platform['kind'] == 'CONTRIBUTION') & (df_platform['paymentMethod.service'] == 'STRIPE')]

In [164]:
df_platform_stripe.head()

,id,uuid,kind,paymentMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31T22:46:58.403Z,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,f3fb4940-1026-4440-bd85-bc3c0cb439b0,f3fb4940-1026-4440-bd85-bc3c0cb439b0,CONTRIBUTION,NaN,NaN,9931763a-79b1-4141-923f-216db174f56e,CREDIT,Financial contribution to OpenSCAD,2023-12-31T22:29:44.179Z,ch_3OTXVMBYycQg1OMf0qVZ5tjm,False,False,NaN,False,NaN,Credit,USD,5000,Amount,USD,5000,Amount,USD,4825,Amount,USD,4825,Amount,USD,-175,Amount,rmvrwng4-kj03dpbk-ljzpz57o-yl9e8xba,132610.0,OpenSCAD,openscad,False,COLLECTIVE,Collective,n4gx0bro-v5m96n0l-878qd8lk-3ey7jzwa,467683,Holland Hopson,guest-8761eb54,False,INDIVIDUAL,Individual,ywz9j4av-god8pg8g-8ly6mr35-nxklb0e7,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,e438b64b-473b-405b-be1c-67c7cfc58fba,e438b64b-473b-405b-be1c-67c7cfc58fba,CONTRIBUTION,NaN,NaN,91d79b1b-e38d-4559-be03-ad4a5f5fa301,CREDIT,Monthly financial contribution to LibreELEC,2023-12-31T19:06:49.150Z,ch_3OTULSBYycQg1OMf10wlDceK,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,926,Amount,USD,926,Amount,USD,-74,Amount,88rzownx-l9e50pxv-3vb6ymvb-dgk7j43a,204839.0,LibreELEC,libreelec,False,COLLECTIVE,Collective,e0adkjrv-8xwm69oy-zwl67z5n-4l93bgoy,744470,Daniel Dolejska,daniel-dolejska,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzez-e7bpbma8-37xlw95j,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,CONTRIBUTION,NaN,NaN,0870251c-c506-421d-b969-9b98f303bb90,CREDIT,Financial contribution to Spotube,2023-12-31T19:03:20.581Z,ch_3OTUG8BYycQg1OMf0FRuZULZ,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,3kzxy4v0-7wlr6mvy-5m76mj9n-o8agdbe5,475166.0,Spotube,spotube,False,COLLECTIVE,Collective,zaxon793-jy8gpl0r-4k9pbrkd-emwl5v04,744469,Ignacho,guest-a4555b91,False,INDI

In [165]:
# for each row in df_stripe, find the corresponding row in df_platform, match df_stripe['id'] with df_platform['merchantId']
# if the row exists, add the platform transaction id to df_stripe in a new column called platform_transaction_id
# if the row does not exist, set the platform_transaction_id to None

# Merge the two dataframes on the 'id' from df_stripe and 'merchantId' from df_platform
merged_df = pd.merge(df_stripe_reconciliation, df_platform[['merchantId', 'id']], left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_stripe_reconciliation = merged_df

In [175]:
# count the number of Stripe transactions that have a id_platform_transaction
print(f"Stripe transactions that have a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])}")

# count the number of Stripe transactions that do not have a id_platform_transaction
print(f"Stripe transactions without a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()])}")

# print the percentage of Stripe transactions that have a corresponding platform transaction
print(f"Percentage of Stripe transactions that have a corresponding platform transaction: {round(len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])/len(df_stripe_reconciliation)*100, 2)}%")

Stripe transactions that have a corresponding platform transaction: 131588
Stripe transactions without a corresponding platform transaction: 432
Percentage of Stripe transactions that have a corresponding platform transaction: 99.67%
